In [1]:
import ee
import geemap

In [4]:
ee.Initialize()

In [5]:
aoi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na','Kenya')).geometry()

In [6]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10 #bitwise left shift
    cirrusBitMask = 1 << 11 
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).copyProperties(image)

In [7]:
def getEVI(image):
    EVI = image.expression(
        '2.5 * ((NIR-RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR' : image.select('B8').divide(10000),
            'RED' : image.select('B4').divide(10000),
            'BLUE' : image.select('B2').divide(10000),
        }).rename("EVI")
    image = image.addBands(EVI)
    return(image)

In [11]:
def addDate(image):

    img_date = ee.Date(image.date())

    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))

    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [20]:
sentinel_img2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(aoi) \
    .filterDate('2022-05-01','2022-10-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 34)).median()

image2 = sentinel_img2.select('B4', 'B3', 'B2')

In [21]:
map1 = geemap.Map()
map1.addLayer(image2.clip(aoi),{'min':100,'max':3000},"yoh")
map1

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [30]:
# sentinel_data = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
#     .filterBounds(aoi) \
#     .filterDate('2022-05-01','2022-10-30') \
#     .map(getEVI).map(addDate).map(maskS2clouds).median()

sentinel_data1 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(aoi) \
    .filterDate('2022-05-01','2022-10-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 34)) \
    .map(getEVI).median()


In [32]:
palette = [
    'blue', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301']
# palette=['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
pall = {"min":0, "max":1, 'palette':palette}

map2 = geemap.Map()
map2.centerObject(aoi, 6)
map2.addLayer(sentinel_data1.clip(aoi).select('EVI'),pall,"yooh1")
map2

Map(center=[0.5578063517900957, 37.84559934447033], controls=(WidgetControl(options=['position', 'transparent_…